Copyright (c) Microsoft Corporation.  
Licensed under the MIT License. 

# Summarization Evaluation
This notebook explains the metrics commonly used to evaluate text summarization results and how to use the evaluation utilities provided in the repo. 

## ROUGE
Recall-Oriented Understudy for Gisting Evaluation(ROUGE) is a set of metrics for evaluating automatic text summarization and machine translation results. The metrics compare machine-generated summaries or translations against one or multiple reference summaries or translations created by human.  
Commonly used ROUGE metrics are ROUGE-1, ROUGE-2, and ROUGE-L
* ROUGE-1: Overlap of unigrams (single words) between machine-generated and reference summaries. 
* ROUGE-2: Overlap of bigrams (two adjacent words) between machine-generated and reference summaries.
* ROUGE-L: Longest Common Subsequence (LCS), which doesn't require consecutive matches but in-sequence matches that refect sentence-level structure similarity.  

For each metric, recall, precision, and F1 score are computed. 

**Utilities for computing ROUGE**
* `compute_rouge_perl`: The [pyrouge](https://github.com/bheinzerling/pyrouge) package based on the ROUGE package written in perl is the most popular package for computing ROUGE scores. We provide the `compute_rouge_perl` function based on pyrouge. This function supports English only. 
* `compute_rouge_python`: The [py-rouge](https://pypi.org/project/py-rouge/) package is a Python implementation of the ROUGE metric which produces almost the same results as the perl implemenation. Since it's easier to install than pyrouge and can be extended to other languages, we provide the `compute_rouge_python` function based on py-rouge. Currently, English and Hindi are supported. Supports for other languages will be added on an as-needed basis.

In [ ]:
import os
import sys

nlp_path = os.path.abspath('../../')
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)
    
from utils_nlp.eval import compute_rouge_perl, compute_rouge_python

### Sample inputs
Both `compute_rouge_perl` and `compute_rouge_python` takes lists of candidate summaries and reference summaries as inputs. Alternatively, you can also provide paths to files containing the candidates and references and set the `is_input_files` argument to `True`. 

In [ ]:
summary_candidates = ["The stock market is doing well this year.", "The movie is very popular."]
summary_references = ["The stock market is doing really well in 2019.", "The movie is very popular among millennials."]

### compute_rouge_python
To use `compute_rouge_python`, you only need to install the Python package `py-rouge` and `nltk`.

In [ ]:
python_rouge_scores = compute_rouge_python(cand=summary_candidates, ref=summary_references)

In [ ]:
print("ROUGE-1: {}".format(python_rouge_scores["rouge-1"]))
print("ROUGE-2: {}".format(python_rouge_scores["rouge-2"]))
print("ROUGE-L: {}".format(python_rouge_scores["rouge-l"]))

The `compute_rouge_python` function can also support non-English languages. Currently, only Hindi is supported. Support for other languages will be added on an as-needed basis.  
Note that the Hindi sample inputs are generated by translation, so they are not perfect, but suffcient for testing.  

In [ ]:
summary_candidates_hi = [
    "शेयर बाजार इस साल बहुत अच्छा कर रहा है। 2020 के लिए भी यही उम्मीद है।",
    "नई फिल्म बहुत लोकप्रिय है।",
]
summary_references_hi = [
    "शेयर बाजार 2019 में वास्तव में अच्छा कर रहा है। आशा है कि 2020 भी ऐसा ही होगा।",
    "फिल्म सदियों के बीच बहुत लोकप्रिय है।",
]

In [ ]:
python_rouge_scores_hi = compute_rouge_python(cand=summary_candidates_hi, ref=summary_references_hi, language="hi")
print("ROUGE-1: {}".format(python_rouge_scores_hi["rouge-1"]))
print("ROUGE-2: {}".format(python_rouge_scores_hi["rouge-2"]))
print("ROUGE-L: {}".format(python_rouge_scores_hi["rouge-l"]))

### compute_rouge_perl
To use `compute_rouge_perl`, in addition to installing the Python package `pyrouge`, you also need to go through the following setup steps on a Linux machine.  
**NOTE**: Set `PYTHON_PATH` to the root directory of the conda environment where you installed `pyrouge` first. You can use "conda env list" to find the`PYTHON_PATH`. 

In [ ]:
python_path = !which python

In [ ]:
PYTHON_PATH = "/".join(python_path[0].split(os.pathsep)[0].split("/")[0:-2])

In [ ]:
%%bash
git clone https://github.com/andersjo/pyrouge.git
PYROUGE_PATH=./pyrouge 
PYTHON_PATH=$PYTHON_PATH #<root directory of conda environment> #e.g./data/anaconda/envs/nlp_gpu
$PYTHON_PATH/bin/pyrouge_set_rouge_path $PYROUGE_PATH/tools/ROUGE-1.5.5

# install XML::DOM plugin, instructions https://web.archive.org/web/20171107220839/www.summarizerman.com/post/42675198985/figuring-out-rouge
sudo cpan App::cpanminus
sudo cpanm XML::DOM

# install XLM::Parser and its dependencies
sudo apt-get update
sudo apt-get install libexpat1-dev -y
sudo cpanm  XML::Parser

# Fix WordNet issue
# Instructions https://web.archive.org/web/20180812011301/http://kavita-ganesan.com/rouge-howto/#IamHavingWordNetExceptions
cd  $PYROUGE_PATH/tools/ROUGE-1.5.5/data/
rm WordNet-2.0.exc.db

cd WordNet-2.0-Exceptions/
./buildExeptionDB.pl . exc WordNet-2.0.exc.db
cd ..
ln -s WordNet-2.0-Exceptions/WordNet-2.0.exc.db WordNet-2.0.exc.db

In [ ]:
perl_rouge_scores = compute_rouge_perl(cand=summary_candidates, ref=summary_references)

In [ ]:
perl_rouge_scores

For each score, the 95% confidence interval is also computed, i.e. "\_cb" and "\_ce" stand for the beginning  and end of the confidence interval, respectively.  
In addition to ROUGE-1, ROUGE-2, ROUGE-L, the perl script computes a few other ROUGE scores. See details of all scores [here](https://en.wikipedia.org/wiki/ROUGE_%28metric%29).  